<a href="https://colab.research.google.com/github/EliEli94/EliEli94/blob/main/idinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-j6461hvt
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-j6461hvt
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp310-cp310-linux_x86_64.whl size=375615 sha256=a9ef95ab9a42cfb9add3dcf97e2be6b79d32a71bc7bf2648c323372b0817ed2f
  Stored in directory: /tmp/pip-ephem-wheel-cache-wb39nxyu/wheels/39/61/b4/480fbddb4d3d6bc34083e7397bc6f5d1381f79acc68e9f3511
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.8
    Uninstalling pycocotools-2.0.8:
      Successfully uninstalled pycocotools-2.0.8


In [2]:
%%shell
git clone https://github.com/MbassiJaphet/pytorch-for-information-extraction.git
% cd pytorch-for-information-extraction/code

Cloning into 'pytorch-for-information-extraction'...
remote: Enumerating objects: 1170, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 1170 (delta 6), reused 5 (delta 5), pack-reused 1150
Receiving objects: 100% (1170/1170), 39.14 MiB | 29.62 MiB/s, done.
Resolving deltas: 100% (556/556), done.
/bin/bash: line 2: fg: no job control


CalledProcessError: Command 'git clone https://github.com/MbassiJaphet/pytorch-for-information-extraction.git
% cd pytorch-for-information-extraction/code
' returned non-zero exit status 1.

In [6]:
# Resolve imports for detection module
import os
import cv2
import time
import torch
import torchvision

import numpy as np

import utils

from PIL import Image, ImageDraw, ImageFilter
#%matplotlib inline
from matplotlib import pyplot as plt

# define transforms to convert PIL image to torch.Tensor
imgToTensor = torchvision.transforms.ToTensor()
# define transforms to convert torch.Tensor to PIL image
tensorToPIL = torchvision.transforms.ToPILImage()

In [7]:
class DetectionDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, mode=None, transforms=None):
        self.mode = mode
        self.data_path = data_path
        self.transforms = transforms
        # everything but our dataset classes ibelong to class 'BACKGROUND'
        self.classes = ['BACKGROUND']
        # loading our dataset classes names
        _classes_names = utils.load_json(os.path.join(data_path, 'classes.json'))['classes']
        # implicitly attributing index '0' to BACKGROUND class
        self.classes.extend(_classes_names)
        # load all image files
        dataset_file = os.path.join(data_path, str(mode).lower().__add__('.json'))
        if not os.path.exists(dataset_file):
            raise Exception("Invalid Mode: '{}'\n Available modes are: 'TRAIN', 'VALID', 'TEST'.".format(mode))
        data_dict = utils.load_json(dataset_file)
        self.image_urls = dict()
        self.annotation_urls = dict()
        for object_id, item_dict in enumerate(data_dict['data']):
            self.image_urls[object_id] = item_dict['image_url'].replace('\\', '/')
            self.annotation_urls[object_id] = item_dict['annotation_url'].replace('\\', '/')

    def __getitem__(self, idx):
        # load images and annotations
        image_url = self.image_urls[idx]
        annotation_url = self.annotation_urls[idx]
        annotation_dict = utils.load_json(annotation_url)
        image = Image.open(image_url)
        image_height, image_width = image.size
        num_objects = len(annotation_dict['shapes'])
        labels, boxes, polygons = list(), list(), list()
        target = dict()

        for idx, shape in enumerate(annotation_dict['shapes']):
            label = self.classes.index(shape['label'].upper())
            polygon = [(int(x), int(y)) for x, y in shape['points']]
            labels.append(label)
            polygons.append(polygon)

        masks_array = np.zeros((image_width, image_height))
        masks_array = utils.draw_polygons_on_image_array(masks_array, polygons)
        object_ids = np.unique(masks_array)[1:]  # Remove index for background
        mask_arrays = masks_array == object_ids[:, None, None]

        for mask_array in mask_arrays:
            box = utils.compute_box_from_mask_array(mask_array)
            boxes.append(box)

        labels_tensor = torch.from_numpy(np.array(labels))
        boxes_tensor = torch.as_tensor(boxes, dtype=torch.float32)
        masks_tensor = torch.as_tensor(mask_arrays, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes_tensor[:, 3] - boxes_tensor[:, 1]) * (boxes_tensor[:, 2] - boxes_tensor[:, 0])

        is_crowd = torch.zeros((num_objects,), dtype=torch.int64)

        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = is_crowd
        target["labels"] = labels_tensor
        target["boxes"] = boxes_tensor
        target["masks"] = masks_tensor

        if self.transforms is not None: image_tensor, target = self.transforms(image, target)

        return image_tensor, target

    def __len__(self):
        return len(self.image_urls)

In [8]:
import modules.detection.scripts.transforms as detection_transforms

def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(detection_transforms.ToTensor())
    ### feel free to add additional transforms here below

    return detection_transforms.Compose(transforms)

In [10]:
detection_data_path = os.path.join('datasets', 'detection')
# training dataset
detection_train_set = DetectionDataset(detection_data_path, mode='train', transforms=get_transform(True))
# validation dataset
detection_valid_set = DetectionDataset(detection_data_path, mode='valid', transforms=get_transform(True))
#testing dataset
detection_test_set = DetectionDataset(detection_data_path, mode='test', transforms=get_transform(False))

detection_classes = detection_train_set.classes
num_detection_classes  = len(detection_classes)

In [11]:
print('Number of classes: {}\nClasses: {}'.format(num_detection_classes, detection_classes))

Number of classes: 2
Classes: ['BACKGROUND', 'STUDENT_ID']


In [14]:
id = 0
obj_id = None
obj_label = ''
fig = plt.figure(figsize=(15, 15))

image_tensor, targets = detection_train_set[id]

boxes = targets['boxes'] # retrieve bounding boxes
image = utils.tensorToPIL(image_tensor)
image_array = np.array(image)

for box in boxes :
  cv2.rectangle(image_array, (box[0],box[1]), (box[2],box[3]), (255,0,0), 2) # draw bounding boxes

ax = fig.add_subplot(1, 2, 1, xticks=[], yticks=[])
plt.imshow(Image.fromarray(image_array))
ax.set_title('Image')

if obj_id is not None:
    mask_tensor = targets['masks'][obj_id] # retrieve bounding masks
    obj_label_idx = targets['labels'][[obj_id]].item() # retrieve bounding labels
    obj_label = ': ' + detection_classes[obj_label_idx]
else :
    mask_tensor = torch.zeros_like(image_tensor)
    for _mask_tensor in targets['masks'] : mask_tensor += _mask_tensor # paste mask for every object

ax1 = fig.add_subplot(1, 2, 2, xticks=[], yticks=[])
ax1.set_title('Segentation Mask' + obj_label)
mask = utils.tensorToPIL(mask_tensor)
plt.imshow(mask)
None

error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'rec'. Expected sequence length 4, got 2
>  - Can't parse 'rec'. Expected sequence length 4, got 2


<Figure size 1500x1500 with 0 Axes>

In [15]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_instance_segmentation_model(num_classes, state_dict=None):
    # load an instance segmentation model pre-trained on COCO
    detection_model = torchvision.models.detection.maskrcnn_resnet50_fpn(progress=True, pretrained=True)
    # get the number of input features for the classifier
    in_features = detection_model.roi_heads.box_predictor.cls_score.in_features
    ### FINETUNE pret-trained model
    # replace the pre-trained head with a new one
    detection_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    # now get the number of input features for the mask classifier
    in_features_mask = detection_model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    detection_model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer,  num_classes)
    if state_dict is not None: detection_model.load_state_dict(state_dict)
    return detection_model

In [16]:
# select hardware use for computations
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

load_detection_checkpoint = True
save_detection_checkpoint = False

detection_checkpoint_path = os.path.join('checkpoints', 'detection_mask_rcnn_resnet50.pth.tar')

'''
Do not edit the lines below
'''

if load_detection_checkpoint :
  detection_checkpoint = torch.load(detection_checkpoint_path, map_location=device) if os.path.exists(detection_checkpoint_path) else None
  detection_model_state_dict = detection_checkpoint['model_state_dict'] if not detection_checkpoint == None else None
  detection_optimizer_state_dict = detection_checkpoint['optimizer_state_dict'] if not detection_checkpoint == None else None
else :
  detection_checkpoint, detection_model_state_dict, detection_optimizer_state_dict = None, None, None

if not save_detection_checkpoint : detection_checkpoint_path = None
# initialize detection model using the state dictionary from checkpoint
detection_model = get_instance_segmentation_model(num_detection_classes, state_dict = detection_model_state_dict)

if detection_checkpoint == None : print('No checkpoint loaded ! Loaded pre-trained model instead...')
else :
  print('Loaded model from checkpoint...')
  utils.checkpoint_summary(detection_checkpoint)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:01<00:00, 131MB/s]


No checkpoint loaded ! Loaded pre-trained model instead...


In [17]:
import modules.detection.scripts.utils as script_utils

# data loader for training
detection_train_loader = torch.utils.data.DataLoader(
    detection_train_set, batch_size=6, shuffle=True, num_workers=4,
    collate_fn=script_utils.collate_fn)

# data loader for validation
detection_valid_loader = torch.utils.data.DataLoader(
    detection_valid_set, batch_size=2, shuffle=False, num_workers=2,
    collate_fn=script_utils.collate_fn)

# data loader for testing
detection_test_loader = torch.utils.data.DataLoader(
    detection_test_set, batch_size=2, shuffle=False, num_workers=0,
    collate_fn=script_utils.collate_fn)

# defining orientation data loaders dictionary
detection_loaders = {
    'train' : detection_train_loader,
    'valid' : detection_valid_loader,
    'test' : detection_test_loader,
}

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [19]:
# activate gradients calculation for unfreezed parameters
detection_params = [p for p in detection_model.parameters() if p.requires_grad]
# initialize training optimizer with learning rate, momentum and weight decay
detection_optimizer = torch.optim.SGD(detection_params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# define learning rate schelduler to gradually decay learning rate
detection_lr_scheduler = torch.optim.lr_scheduler.StepLR(detection_optimizer, step_size=10, gamma=0.95)

# load optimizer state dictionary from checkpoint if available
if detection_optimizer_state_dict is None:  print('No checkpoint loaded ! Optimizer not loaded from checkpoint...')
else:
    detection_optimizer.load_state_dict(detection_optimizer_state_dict)
    print('Loaded optizer from checkpoint...')

No checkpoint loaded ! Optimizer not loaded from checkpoint...


In [23]:
!pip uninstall -y apex

In [25]:
# import training and evaluation functions
from modules.detection.scripts.engine import train_one_epoch, evaluate

def train_detection_model(model, num_epochs=10, loaders=None, checkpoint=None, checkpoint_path=None,
                optimizer= None, lr_scheduler= None, print_freq=1, device=torch.device('cuda')):

    if checkpoint is None: start_epoch = 1
    else:
        print('Resuming training from checkpoint...')
        start_epoch = checkpoint['epoch'] + 1

    model.to(device) # Move model to cpu or cuda device

    time_train = time.time()
    for epoch in range(start_epoch, num_epochs + 1):
        time_epoch = time.time()
        # train for one epoch, printing every '{print_freq}' iterations
        train_one_epoch(model, optimizer, loaders['train'], device, epoch, print_freq=print_freq)
        # update the learning rate
        lr_scheduler.step()
        # evaluate model on the validation dataset
        # evaluate(model, loaders['valid'], device=device)

        time_epoch_elapsed = time.time() - time_epoch
        time_train_elapsed = time.time() - time_train
        print('Epoch: {}\tEpoch Time: {:.0f}m {:.0f}s\tElapsed Time: {:.0f}m {:.0f}s'.format(
             epoch, time_epoch_elapsed // 60, time_epoch_elapsed % 60,
             time_train_elapsed // 60, time_train_elapsed % 60))

        # Save checkpoint after every epoch if checkpoint_path is given
        if not checkpoint_path == None:
            utils.save_checkpoint(model.state_dict(), optimizer.state_dict(), epoch, checkpoint_path)

    return model # retun trained model

In [ ]:
# start training the detection model for 20 epochs
detection_model = train_detection_model(detection_model, num_epochs= 20, loaders= detection_loaders,
                        checkpoint= detection_checkpoint, checkpoint_path= detection_checkpoint_path,
                        optimizer= detection_optimizer, lr_scheduler= detection_lr_scheduler,
                        print_freq=10, device= device)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: [1]  [ 0/20]  eta: 1:14:40  lr: 0.005000  loss: 4.8553 (4.8553)  loss_classifier: 0.5772 (0.5772)  loss_box_reg: 0.1257 (0.1257)  loss_mask: 4.1330 (4.1330)  loss_objectness: 0.0058 (0.0058)  loss_rpn_box_reg: 0.0135 (0.0135)  time: 224.0260  data: 1.5236
Epoch: [1]  [10/20]  eta: 0:35:55  lr: 0.005000  loss: 0.9627 (1.5756)  loss_classifier: 0.0807 (0.1431)  loss_box_reg: 0.1325 (0.1379)  loss_mask: 0.6950 (1.2705)  loss_objectness: 0.0127 (0.0129)  loss_rpn_box_reg: 0.0107 (0.0112)  time: 215.5683  data: 0.1525
